In [3]:
import numpy as np
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
%matplotlib inline

For simplicity, assume a layer of constant composition, density and thickness. The total volume $V_X$ of precipitate $X$ is obtained from $\rho_X = M_X/V_X$, where $\rho_X$ is density and $M_X$ the total mass of precipitate. 

The total mass fraction of precipitate is 

$$ C_X = \frac{M_X}{M_{tot}} = \int \frac{d C_X}{d t} dt = \int \frac{d C_X}{d T} \frac{d T}{d t}dt$$

where $M_{tot}$ is the total core mass, $t$ is time and $T$ is temperature. 

Assuming $d C_X /d T$ and $d T/d t$ are constant gives

$$ \frac{M_X}{M_{tot}} = \frac{d C_X}{d T} \frac{d T}{d t} t_{p}$$

where $t_p$ is the time over which precipitation has occurred. The first assumption is suggested by the results of Badro et a, 2018. The latter is not so good. 

The volume of precipitate

$$ V_X = \frac{4}{3} \pi ( r_{c}^3 - r_p^3) \Rightarrow r_p^3 = r_c^3 - \frac{3V_X}{4\pi}$$

In [12]:
dcMgOdT = 3e-6 # From Badro et al 2018, fig 3b
dTdt    = 100.0# K/Gyr cooling rate
tp      = 4.5  # Assume precipitation has occurredd since core formation
Mc      = 1.94e24
rhoMgO  = 5000.0
rc      = 3480e3
secingyr= 60*60*24*365*1e9

MMgO    = dcMgOdT * dTdt * tp * Mc
VMgO    = MMgO/rhoMgO
rp      = ( rc**3 - (3*VMgO/(4*np.pi)) )**0.33333333 

print('Mass, Volume, rp (km), layer thickness (km) {:10.5e} {:10.5e} {:10.5f} {:10.5f}'.format(MMgO, VMgO, rp, (rc-rp)/1e3))


Mass, Volume, rp (km), layer thickness (km) 2.61900e+21 5.23800e+17 3476554.17532    3.44582


In [16]:
# Now use output from a thermal history model

import pickle
import numpy as np
import matplotlib.pyplot as plt

filename = 'drho=600_k=natgeo_Cm=3.0e-06.pik'
data = pickle.load(open(filename,'rb'))

#Time in Gyrs
time = data['core']['time']/(1e6*data['parameters']['ys']) / 1e3
time = np.squeeze(time) #change dimension from (x,1) to (x,)
dt   = time[1:] -  time[0:-1]

#CMB temperature
Tc   = np.squeeze(data['core']['T_cmb'])

#CMB cooling rate
dTcmb_dt = np.squeeze(data['core']['dT_dt'] * (Tc/data['core']['Tcen'])) * secingyr

In [26]:
MMgO_run  = dcMgOdT * Mc * np.trapz(dTcmb_dt, time, dt)
VMgO_run  = MMgO_run/rhoMgO

rp_run    = ( rc**3 - (3*VMgO_run[-1]/(4*np.pi)) )**0.33333333 

print('Mass, Volume, rp (km), layer thickness (km) {:10.5e} {:10.5e} {:10.5f} {:10.5f}'
      .format(MMgO_run[-1], VMgO_run[-1], rp_run, (rc-rp_run)/1e3))


7.73476263652578e+21 1.5469525273051558e+18
Mass, Volume, rp (km), layer thickness (km) 7.73476e+21 1.54695e+18 3469804.61087   10.19539
